https://www.unitstatistics.com/starcraft2/ used for getting base unit Hp, Dps etc.

Based on the data the following information was used for the units:

| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |




In [236]:
from IPython.display import Markdown

table = """
| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |
"""

display(Markdown(table))


| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |


In [237]:
import copy
from itertools import product

In [238]:
class CurrentBoard:
    def __init__(self, width=30, height=10, marine_count=4, zergling_count=6):
        self.width = width
        self.height = height
        self.board = [[None for _ in range(width)] for _ in range(height)]
        self.marines = []  # List to store Marine objects
        self.zerglings = []  # List to store Zergling objects
        self.center_x = width // 2
        self.center_y = height // 2
        self.marine_count = marine_count
        self.zergling_count = zergling_count
        self.state = self.state_of_board()


    def is_in_melee_combat(self, unit):
        # Check if unit is adjacent to an enemy unit
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'

        # Check adjacent squares
        directions = [
            (0, 1),  # Up
            (0, -1),  # Down
            (1, 0),  # Right
            (-1, 0),  # Left
            (1, 1),  # Up-Right
            (1, -1),  # Down-Right
            (-1, 1),  # Up-Left
            (-1, -1)  # Down-Left
        ]

        for dx, dy in directions:
            check_x = unit.x + dx
            check_y = unit.y + dy

            # Check if the adjacent square is within board bounds
            if (0 <= check_x < self.width and
                    0 <= check_y < self.height):
                adjacent_unit = self.board[check_y][check_x]
                # If there's an enemy unit in an adjacent square
                if (adjacent_unit is not None and
                        adjacent_unit.team == enemy_team):
                    return True

        return False

    def get_attackable_targets(self, unit):
        targets = []
        # Determine targets based on unit type and attack range
        enemy_list = self.zerglings if unit.team == 'Marine' else self.marines
        for potential_target in enemy_list:
            # Calculate Manhattan distance
            distance = abs(unit.x - potential_target.x) + abs(unit.y - potential_target.y)
            if distance <= unit.attack_range+0.2:
                targets.append(potential_target)

        return targets

    def get_possible_actions(self, unit):
        # Check if unit can attack and is not in melee combat (indicating a ranged unit)
        attackable_targets = self.get_attackable_targets(unit)
        #check if in melee
        in_melee_combat = self.is_in_melee_combat(unit)
        # Get possible moves
        possible_moves = self.get_possible_moves(unit)
        # Retreat actions:
        retreat_directions = self.get_retreat_directions(unit)

        # Detailed action generation logic
        actions = []

        # First, check for attack actions
        if attackable_targets and not in_melee_combat:
            attack_actions = [('attack', target) for target in attackable_targets]
            retreat_actions = [('retreat', direction) for direction in retreat_directions]
            actions.extend(attack_actions)
            actions.extend(retreat_actions)

        # If in melee combat, prioritize attack
        if in_melee_combat:
            attack_actions = [('attack', target) for target in attackable_targets]
            actions.extend(attack_actions)

        # Add move actions if no attacks or not in melee
        if not actions:
            move_actions = [('move', move) for move in possible_moves]
            actions.extend(move_actions)

        # Add retreat actions if no other actions
        if not actions:
            retreat_actions = [('retreat', direction) for direction in retreat_directions]
            actions.extend(retreat_actions)

        return actions




    def get_possible_moves(self, unit):
        """
        Generate possible moves dynamically based on nearest enemy
        """
        moves = []

        # Find the nearest enemy to determine movement direction
        nearest_enemy = self.find_nearest_enemy(unit)

        if nearest_enemy is None:
            # If no enemy found, return current position
            return [(unit.x, unit.y)]

        # Determine primary movement direction based on enemy position
        dx = 1 if nearest_enemy.x > unit.x else -1
        dy = 1 if nearest_enemy.y > unit.y else -1

        # Define directional priority based on enemy location
        if unit.team == 'Zergling':
            directions = [
                (dx, 0),  # Primary horizontal movement towards enemy
                (dx, dy),  # Diagonal movement towards enemy
                (dx, -dy),  # Alternate diagonal
                # (-dx, 0)  # Backward movement if needed
            ]
        else:  # Marine
            directions = [
                (-dx, 0),  # Primary horizontal movement towards enemy (reversed for Marines)
                (-dx, dy),  # Diagonal movement towards enemy
                (-dx, -dy),  # Alternate diagonal
                (dx, 0)  # Backward movement if needed
            ]

        for move_dx, move_dy in directions:
            # Calculate end position based on unit's full speed
            move_x = unit.x + move_dx * unit.speed
            move_y = unit.y + move_dy * unit.speed

            # Check if move is within board boundaries
            if (0 <= move_x < self.width and 0 <= move_y < self.height):
                # Get the unit at the destination tile
                dest_unit = self.board[move_y][move_x]

                # If destination is occupied
                if dest_unit is not None:
                    # Check if the occupying unit is an enemy
                    if dest_unit.team != unit.team:
                        # Calculate the tile just before the occupied tile
                        pre_move_x = unit.x + move_dx * (unit.speed - 1)
                        pre_move_y = unit.y + move_dy * (unit.speed - 1)

                        # Check if pre-move tile is within bounds and unoccupied
                        if (0 <= pre_move_x < self.width and
                                0 <= pre_move_y < self.height and
                                self.board[pre_move_y][pre_move_x] is None):
                            moves.append((pre_move_x, pre_move_y))
                else:
                    # Destination is unoccupied, add full move
                    moves.append((move_x, move_y))

        if not moves:
            moves = [(unit.x, unit.y)]
        return moves


    def find_nearest_enemy(self, unit):
        """
        Find the nearest enemy unit to the given unit
        """
        enemy_team = 'Marine' if unit.team == 'Zergling' else 'Zergling'
        enemy_units = self.marines if enemy_team == 'Marine' else self.zerglings

        if not enemy_units:
            return None

        # Calculate Manhattan distance to each enemy
        enemies_with_distance = [
            (enemy, abs(unit.x - enemy.x) + abs(unit.y - enemy.y))
            for enemy in enemy_units
        ]

        # Find the enemy with the minimum distance
        return min(enemies_with_distance, key=lambda x: x[1])[0]



    def get_retreat_directions(self, unit):
        """
        Generate possible retreat moves with:
        - Unit-specific directional restrictions
        - Board boundary checks
        - Occupied tile checks
        - stay in place option
        """
        retreats = [(unit.x, unit.y)]


        # Determine retreat direction based on unit type
        if unit.team == 'Zergling':
            dx, dy = -1, 0  # Left (backward)
        else:  # Marine
            dx, dy = 1, 0  # Right (backward)

        # Calculate end retreat position based on unit's full speed
        retreat_x = unit.x + dx * unit.speed
        retreat_y = unit.y + dy * unit.speed

        # Check if retreat is within board boundaries
        if (0 <= retreat_x < self.width and 0 <= retreat_y < self.height):
            # If destination is occupied, check the tile just before
            if self.board[retreat_y][retreat_x] is not None:
                # Calculate the tile just before the occupied tile
                pre_retreat_x = unit.x + dx * (unit.speed - 1)
                pre_retreat_y = unit.y + dy * (unit.speed - 1)

                # Check if pre-retreat tile is within bounds and unoccupied
                if (0 <= pre_retreat_x < self.width and
                        0 <= pre_retreat_y < self.height and
                        self.board[pre_retreat_y][pre_retreat_x] is None):
                    retreats.append((pre_retreat_x, pre_retreat_y))
            else:
                # Destination is unoccupied, add full retreat
                retreats.append((retreat_x, retreat_y))

        return retreats


    def get_team_possible_actions(self, team):
        """
        Generate all possible action combinations for a team
        Ensures every unit takes exactly one action
        """
        # Select units based on team
        units = self.marines if team == 'Marine' else self.zerglings

        # Will store possible actions for each unit
        team_unit_actions = []

        # Generate possible actions for each unit
        for unit in units:
            # Get possible actions for this specific unit
            unit_actions = self.get_possible_actions(unit)
            # If no actions are possible, this is an invalid state
            if not unit_actions:
                if unit.hp > 0:
                    print(f"targets in range: {self.get_attackable_targets(unit)}" )
                    print(f"Unit is in melee range? {unit.engaged_in_melee}")
                    print(f"unit possible moves: {self.get_possible_moves(unit)}")
                    print(f"unit possible retreat: {self.get_retreat_directions(unit)}")
                    print(f"Unit actions: {unit_actions}")
                    print(f"Unit hp: {unit.hp}")
                    raise ValueError(f"Unit {unit.team} at ({unit.x}, {unit.y}) has no possible actions")

            team_unit_actions.append(unit_actions)

        # Generate unique action combinations
        return list(set(tuple(combo) for combo in product(*team_unit_actions)))






    def setup_initial_board(self):
        """
        Set up initial board with units at the exact map edges
        """
        # Clear existing units
        self.marines.clear()
        self.zerglings.clear()
        self.board = [[None for _ in range(self.width)] for _ in range(self.height)]

        # Calculate vertical center of the board
        center_y = self.height // 2

        # Calculate front and back row counts
        zergling_front_count = self.zergling_count // 2
        zergling_back_count = self.zergling_count - zergling_front_count

        marine_front_count = self.marine_count // 2
        marine_back_count = self.marine_count - marine_front_count

        # Zerglings formation (left edge)
        front_zergling_x = 0  # Leftmost edge of the map
        back_zergling_x = 1  # One step from the left edge

        # Marines formation (right edge)
        front_marine_x = self.width - 1  # Rightmost edge of the map
        back_marine_x = self.width - 2  # One step from the right edge

        # Vertical positioning for Zerglings
        zergling_front_rows = [
            y for y in range(
                center_y - zergling_front_count // 2,
                center_y + (zergling_front_count + 1) // 2
            )
        ]
        zergling_back_rows = [
            y for y in range(
                center_y - zergling_back_count // 2,
                center_y + (zergling_back_count + 1) // 2
            )
        ]

        # Vertical positioning for Marines
        marine_front_rows = [
            y for y in range(
                center_y - marine_front_count // 2,
                center_y + (marine_front_count + 1) // 2
            )
        ]
        marine_back_rows = [
            y for y in range(
                center_y - marine_back_count // 2,
                center_y + (marine_back_count + 1) // 2
            )
        ]

        # Place Zerglings in front rows
        for y in zergling_front_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(front_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][front_zergling_x] = zergling

        # Place Zerglings in back rows
        for y in zergling_back_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(back_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][back_zergling_x] = zergling

        # Place Marines in front rows
        for y in marine_front_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(front_marine_x, y)
                self.marines.append(marine)
                self.board[y][front_marine_x] = marine

        # Place Marines in back rows
        for y in marine_back_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(back_marine_x, y)
                self.marines.append(marine)
                self.board[y][back_marine_x] = marine

        # Verify unit counts
        assert len(self.zerglings) == self.zergling_count, f"Zergling count is {len(self.zerglings)}, should be {self.zergling_count}"
        assert len(self.marines) == self.marine_count, f"Marine count is {len(self.marines)}, should be {self.marine_count}"
        self.state = self.state_of_board()


    def move_unit(self, unit, target):
        # Remove unit from current position
        self.board[unit.y][unit.x] = None

        # Update unit's coordinates
        new_x, new_y = target
        unit.x = new_x
        unit.y = new_y

        # Place unit in new position
        self.board[new_y][new_x] = unit


    def attack_unit(self, attacker, target_unit):
        # Find the actual target unit on the board at the given coordinates
        actual_target = self.board[target_unit.y][target_unit.x]

        if actual_target is None:
            return  # No unit to attack

        # Calculate damage based on attacker's characteristics
        damage = attacker.damage

        # Reduce target's health
        actual_target.hp -= damage

        # Check if target is destroyed
        if actual_target.hp <= 0:
            # Remove target from board and unit list
            self.board[actual_target.y][actual_target.x] = None

            if actual_target.team == 'Marine':
                # Find and remove the unit with matching coordinates and team
                self.marines = [marine for marine in self.marines
                                if not (marine.x == actual_target.x and
                                        marine.y == actual_target.y and
                                        marine.team == actual_target.team)]
            else:
                # Same for Zerglings
                self.zerglings = [zergling for zergling in self.zerglings
                                  if not (zergling.x == actual_target.x and
                                          zergling.y == actual_target.y and
                                          zergling.team == actual_target.team)]



    def display_board(self):
        """
        Visualize the current board state with precise unit placement
        """
        # Create a copy of the board to visualize
        display_grid = [[' .' for _ in range(self.width)] for _ in range(self.height)]

        # Print the board with detailed unit placement
        print("\nDebug - Unit Positions:")
        for zergling in self.zerglings:
            print(f"Zergling: x={zergling.x}, y={zergling.y} hp={zergling.hp}")
            display_grid[zergling.y][zergling.x] = ' Z'

        for marine in self.marines:
            print(f"Marine: x={marine.x}, y={marine.y} hp={marine.hp}")
            display_grid[marine.y][marine.x] = ' M'

        # Print the board
        for row in display_grid:
            print(''.join(row))

        # Print additional unit information
        print(f"\nMarines: {len(self.marines)}, Zerglings: {len(self.zerglings)}")


    def state_of_board(self):
        # Check if all Marines are dead
        if len(self.marines) == 0:
            return "Zergling"

        # Check if all Zerglings are dead
        if len(self.zerglings) == 0:
            return "Marine"

        # Check if board is full or max turns reached
        if (len(self.marines) + len(self.zerglings) >= self.width * self.height or
                len(self.marines) == 0 or
                len(self.zerglings) == 0):
            return "Draw"

        # Game is still ongoing
        return "U"  # Undetermined/Unfinished

In [239]:
class Unit:
    def __init__(self, x, y, hp, speed, damage, attack_range, team):
        self.x = x
        self.y = y
        self.hp = hp
        self.speed = speed
        self.damage = damage
        self.attack_range = attack_range
        self.team = team  # 'Marine' or 'Zerg'
        self.engaged_in_melee = False



class Marine(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (9.8)
        super().__init__(x, y, hp=45, speed=3, damage=9.8, attack_range=5, team='Marine')


class Zergling(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (10)
        super().__init__(x, y, hp=35, speed=4, damage=10, attack_range=1, team='Zergling')



In [240]:
# Create a board
board = CurrentBoard(width=30, height=10,zergling_count=6, marine_count=3)

# Initial setup will be automatic by calling setup_initial_board()
board.setup_initial_board()

# Display the initial board state
board.display_board()

# was having issues with the board set up, display and debug helped to show this better.



Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6


In [241]:
class SearchTreeNode:
    def __init__(self, current_board, playing_team, depth=0):
        self.current_board = current_board
        self.playing_team = playing_team
        self.value_is_assigned = False
        self.depth = depth
        self.children = []
        self.value = None

    def is_game_over(self):
        # Check if game is over based on board state
        game_state = self.current_board.state_of_board()
        return game_state != "U"

    def evaluate_position(self):
        # Comprehensive board evaluation
        if self.is_game_over():
            # Terminal state evaluation
            game_state = self.current_board.state_of_board()
            if game_state == "Marine Wins":
                return float('inf')
            elif game_state == "Zergling Wins":
                return float('-inf')
            else:
                return 0

        # Multifactor evaluation
        def calculate_unit_score(units, enemy_units):
            return sum([
                unit.hp +  # Health weight
                calculate_positioning_bonus(unit, enemy_units)  # Positioning bonus
                for unit in units
            ])

        def calculate_positioning_bonus(unit, enemy_units):
            # Prevent division by zero by adding a small constant
            epsilon = 1e-6

            # Calculate minimum distance to any enemy unit
            min_distance = min(
                (abs(unit.x - enemy.x) + abs(unit.y - enemy.y))
                for enemy in enemy_units
            ) + epsilon
            # Invert distance for bonus (closer to enemies = higher bonus)
            # Multiply by a scaling factor to control bonus impact
            return (1 / min_distance) * 10


        marine_score = calculate_unit_score(self.current_board.marines, self.current_board.zerglings)
        zergling_score = calculate_unit_score(self.current_board.zerglings, self.current_board.marines)

        # Positive score favors Marines, negative favors Zerglings
        return marine_score - zergling_score

    def generate_children(self):
        # Generate possible move sets for the current team
        team_action_combinations = self.current_board.get_team_possible_actions(self.playing_team)
        # Create child nodes for each action combination
        for action_combination in team_action_combinations:
            # Create a deep copy of the current board
            new_board = copy.deepcopy(self.current_board)

            # Select units for the current team
            units = new_board.marines if self.playing_team == 'Marine' else new_board.zerglings

            # Apply moves to the board
            for unit, (action_type, target) in zip(units, action_combination):
                if action_type == 'move':
                    new_board.move_unit(unit, target)
                elif action_type == 'attack':
                    new_board.attack_unit(unit, target)

            # Create child node with opposite team
            child_team = 'Zergling' if self.playing_team == 'Marine' else 'Marine'
            child = SearchTreeNode(
                new_board,
                child_team,
                depth=self.depth + 1,
            )

            self.children.append(child)


    def create_board_from_action(self, unit, action):
        # Deep copy of current board
        new_board = copy.deepcopy(self.current_board)

        # Apply the action
        action_type, target = action

        if action_type == 'move':
            # Move unit to new position
            new_board.move_unit(unit, target)
        elif action_type == 'attack':
            # Perform attack
            new_board.attack_unit(unit, target)

        return new_board

    def min_max_value(self, current_depth=0, max_depth=3):
        # If value is already assigned, return it
        if hasattr(self, 'value_is_assigned') and self.value_is_assigned:
            return self.value

        # Check if reached max depth or game is over
        # Use current_depth to track actual depth, not self.depth
        if current_depth >= max_depth or self.is_game_over():
            return self.evaluate_position()

        # Ensure children are generated
        if not self.children:
            self.generate_children()

        # If still no children, return evaluation
        if not self.children:
            return self.evaluate_position()

        # Compute values for children recursively
        child_values = []
        for child in self.children:
            # Pass incremented depth to track recursion depth
            child_value = child.min_max_value(current_depth + 1, max_depth)
            child_values.append(child_value)
            child.value = child_value

        # Sort children based on their values
        self.children = sorted(self.children, key=lambda x: x.value)

        # Select value based on current playing team
        if self.playing_team == 'Marine':
            # Marine tries to maximize, so take the highest value
            self.value = max(child_values) if child_values else self.evaluate_position()
        else:
            # Zerg tries to minimize, so take the lowest value
            self.value = min(child_values) if child_values else self.evaluate_position()

        # Mark value as assigned
        self.value_is_assigned = True
        self.children = sorted(self.children, key=lambda x: x.value)

        return self.value



In [242]:
search_tree = SearchTreeNode(board, 'Marine')


In [243]:
search_tree.current_board.display_board()


Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6


In [244]:
best_move = search_tree.min_max_value(max_depth=2)

In [245]:
search_tree.children[0].value

-76.40692634091377

In [246]:
search_tree.children[-1].value


-76.40692634091377

In [247]:
len(search_tree.children)

1

In [248]:
best_move

-76.40692634091377

In [249]:
search_tree.children[0].current_board.display_board()


Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=26, y=5 hp=45
Marine: x=25, y=4 hp=45
Marine: x=25, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . M . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . M M . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6


In [250]:
def play_StarCraft(width=30, height=10, marine_count=3, zergling_count=6):
    # Ask player to choose starting side
    while True:
        try:
            player_side = input("Choose your side (M for Marine, Z for Zerg): ").upper().strip()

            if player_side == 'M':
                player_side = 'Marine'
                break
            elif player_side == 'Z':
                player_side = 'Zerg'
                break
            else:
                print("Invalid input. Please enter 'M' for Marine or 'Z' for Zerg.")
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Please try again.")

    # Create initial board with specified parameters
    cb = CurrentBoard(width, height, marine_count, zergling_count)
    cb.setup_initial_board()

    # Determine who starts based on player's choice
    if player_side == 'Marine':
        current_player = 'Zergling'  # AI will start with Zerg move
        players_turn = False
    else:
        current_player = 'Marine'
        players_turn = True

    # Game loop
    for _ in range(100):  # Prevent infinite loop
        # Display current board state
        cb.display_board()

        # Prompt to continue
        # input(f"Press Enter to continue {current_player}'s turn...")

        if players_turn:
            # Player's turn
            search_tree = SearchTreeNode(cb, player_side)
            best_move = search_tree.min_max_value(max_depth=2)
            # If player is Marine, select the first child (best move)
            if player_side == 'Marine':
                cb = search_tree.children[-1].current_board
                print(f"Best move Player: {best_move} picked max sorted for player")
            else:  # Zerg
                cb = search_tree.children[0].current_board
                print(f" Best move Player: {best_move} picked min sorted for player")
        else:
            # AI's turn
            search_tree = SearchTreeNode(cb, current_player)
            best_move = search_tree.min_max_value(max_depth=2)

            # If current player is Marine, select the first child (best move)
            if current_player == 'Marine':
                cb = search_tree.children[-1].current_board
                print(f"Best move Enemy: {best_move} picked max sorted for Enemy")
            else:  # Zerg
                cb = search_tree.children[0].current_board
                print(f"Best move Enemy: {best_move} picked min sorted for Enemy")



        # Check game state
        game_state = cb.state_of_board()
        if game_state != "U":
            cb.display_board()
            if game_state == "D":
                print("Draw!")
            else:
                print(f"{game_state} wins!")
            break

        # Switch players
        current_player = 'Zergling' if current_player == 'Marine' else 'Marine'
        players_turn = not players_turn

    print("Game Over")


In [251]:
play_StarCraft()


Debug - Unit Positions:
Zergling: x=0, y=4 hp=35
Zergling: x=0, y=5 hp=35
Zergling: x=0, y=6 hp=35
Zergling: x=1, y=4 hp=35
Zergling: x=1, y=5 hp=35
Zergling: x=1, y=6 hp=35
Marine: x=29, y=5 hp=45
Marine: x=28, y=4 hp=45
Marine: x=28, y=5 hp=45
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 3, Zerglings: 6
Best move Enemy: -76.40692634091377 picked min sorted for Enemy

Debug - Unit Positions:
Zergling: x=4, y=4 hp=35
Zer